In [11]:
from pandas import read_csv
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import operator
# from pandas.plotting import scatter_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler       # scaling data
from sklearn.model_selection import train_test_split # splitting data
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV     # for grid search
from sklearn.pipeline import make_pipeline           # for making pipelines
# from sklearn.neighbors import KNeighborsRegressor    # regressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import AdaBoostRegressor
import warnings
warnings.filterwarnings('ignore')


In [12]:
#read in the en csv files and clean up the column names
all_data = read_csv('data/data.csv')
all_data.isnull().sum()
for col_name in all_data.columns:
    new_col_name = col_name.split(None, 1)[1:]
    new_col_name = ''.join(new_col_name)
    all_data.rename(columns = {col_name:new_col_name[:-33]}, inplace = True)
all_data.columns = all_data.columns.str.replace('-', ' ')
all_data.columns = all_data.columns.str.replace(' ', '_')

all_data = all_data[pd.notnull(all_data['Median_Listing_Price___All_Homes'])]
##### renaming outcome and splitting feature & outcome since we do not want to scale outcomes
all_data = all_data.rename(index=str, columns={"Median_Listing_Price___All_Homes": "outcome"})
outcomes = pd.DataFrame(all_data.outcome)
all_data.drop('outcome', axis=1, inplace=True)
#all_data.dtypes

In [13]:
# Add two new columns that individually record year and month of that row
month = []
year = []
for date in all_data['']:
    month.append(date[0])
    year.append(date[-4:])

all_data['month'] = month
all_data['year'] = year
all_data = all_data.drop('', axis=1)
#all_data = all_data.drop('Zillow_Home_Value_Index_-_Top_Tier_-_Year-Over...', axis=1)

all_data['month'] = all_data['month'].astype('int64')
all_data['year']=all_data['year'].astype('int64')

col_name = all_data.columns

In [41]:
# Handling missing values
from sklearn.impute import SimpleImputer
impu = SimpleImputer()
all_data = pd.DataFrame(impu.fit_transform(all_data))
all_data.columns = col_name
#outcomes.shape
all_data

,Home_Sold_As_Foreclosure___Ratio___All_Homes,Inventory_Measure_(Public),Inventory_Measure___SSA_(Public),Listings_Price_Cuts_(SA)_All_Homes,Listings_Price_Cuts_(SA)_Condominiums,Listings_Price_Cuts_(SA)_Single_Family_Residence,Median_Listing_Price___Condo/Co_op,Median_Listing_Price___Duplex/Triplex,Median_Listing_Price___Five_Or_More_Bedrooms,Median_Listing_Price___Four_Bedrooms,...,Zillow_Home_Value_Index___Top_Tier___Year_Over...,Zillow_Home_Value_Index___Top_Tier,Zillow_Home_Value_Index___Two_Bedrooms,Zillow_Rental_Index___All_Homes_Plus_Multi_Family,Zillow_Rental_Index___All_Homes,Zillow_Rental_Index___Multi_Family_Residence,Zillow_Rental_Index___Single_Family_Residence,Zillow_Rental_Index_Per_Square_Foot___All_Homes,month,year
0,4.745800,2488.000000,2938.00000,10.380988,10.130364,10.475695,319250.0,525000.0,8.866364e+05,599950.0,...,-0.093081,588500.0,327000.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,1.0,2010.0
1,4.718700,2641.000000,3121.00000,10.380988,10.130364,10.475695,319900.0,499950.0,8.866364e+05,599000.0,...,-0.080816,590300.0,327100.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,2.0,2010.0
2,4.728200,3387.000000,3477.00000,10.380988,10.130364,10.475695,306000.0,499950.0,8.866364e+05,596975.0,...,-0.070087,589100.0,327900.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,3.0,2010.0
3,4.720500,3704.000000,3676.00000,10.380988,10.130364,10.475695,299950.0,497000.0,8.866364e+05,579000.0,...,-0.060849,586500.0,328500.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,4.0,2010.0
4,4.460500,3628.000000,3644.00000,10.380988,10.130364,10.475695,299970.0,494000.0,8.866364e+05,589000.0,...,-0.047929,586000.0,327900.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,5.0,2010.0
5,4.462800,3747.000000,3615.00000,10.380988,10.130364,10.475695,299950.0,480000.0,8.866364e+05,569900.0,...,-0.036427,584600.0,326500.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,6.0,2010.0
6,5.655800,3877.000000,3618.00000,10.380988,10.130364,10.475695,299950.0,488900.0,8.866364e+05,559000.0,...,-0.030520,581300.0,323800.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,7.0,2010.0
7,7.171600,3805.000000,3589.00000,10.380988,10.130364,10.475695,299000.0,492500.0,8.866364e+05,559475.0,...,-0.027400,578600.0,322000.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,8.0,2010.0
8,8.697400,3864.000000,3542.00000,17.693415,16.405041,18.216264,289900.0,499000.0,8.866364e+05,559000.0,...,-0.027867,575600.0,320400.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,9.0,2010.0
9,8.182400,3690.000000,3472.00000,14.819889,12.344057,15.776069,285000.0,499000.0,8.866364e+05,550000.0,...,-0.029143,573000.0,318600.0,2131.282828,2180.59596,1843.242424,2268.686869,1.569576,1.0,2010.0


In [29]:
#normalize
from sklearn import preprocessing
# Normalize total_bedrooms column
names = all_data.columns
scaled_data = preprocessing.normalize(all_data)
scaled_data = pd.DataFrame(scaled_data, columns=names)
#scaled_data = scaled_data[(scaled_data > 0).all(1)] ############ What is this for? removed some rows, breaks train split
scaled_data.head()



(109, 88)

In [24]:
####### checking variance, for variance selector we need to specify a threshold (e.g. 0.01) to actually remove features
####### or use along with select percentile
all_data.var().sort_values()

Median_Rental_Price_Per_Square_Foot___Five_Or_...    4.180872e-03
Zillow_Home_Value_Index___Top_Tier___Year_Over...    4.858033e-03
Median_Rental_Price_Per_Square_Foot___Two_Bedr...    5.730616e-03
Zillow_Home_Value_Index___Middle_Tier___Year_O...    6.773786e-03
Zillow_Home_Value_Index___Bottom_Tier_Year_Ove...    1.269311e-02
Median_Rental_Price_Per_Square_Foot___Four_Bed...    3.597748e-02
Zillow_Rental_Index_Per_Square_Foot___All_Homes      5.091846e-02
Median_Rental_Price_Per_Square_Foot___Single_F...    6.082542e-02
Median_Rental_Price_Per_Square_Foot___Three_Be...    7.133456e-02
Median_Rental_Price_Per_Square_Foot___Duplex/T...    7.457217e-02
Median_Rental_Price_Per_Square_Foot___All_Homes      8.068829e-02
Median_Rental_Price_Per_Square_Foot___Condo/Co_op    1.077901e-01
Median_Price_Of_Reduction___Single_Family_Resi...    1.590922e-01
Median_Price_Of_Reduction___All_Homes                1.660078e-01
Median_Rental_Price_Per_Square_Foot___One_Bedroom    1.981635e-01
Median_Ren

In [21]:
#Removing features with low variance
def variance_threshold_selector(data,threshold):
    sel = VarianceThreshold(threshold) 
    sel.fit(data)
    return data[data.columns[sel.get_support(indices=True)]]


var_thrhold = variance_threshold_selector(all_data, threshold=8.082569) 


all_data.shape

(109, 88)

In [40]:
from sklearn.decomposition import PCA
pca = PCA(n_components=7)
principalcomponent=pca.fit(scaled_data)
PCA(copy=True, iterated_power='auto', n_components=7, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

principalcomponent=pca.transform(scaled_data)
principalDf = pd.DataFrame(data = principalcomponent
             , columns = ['principal component 1', 'principal component 2', 'principal component 3', 
                          'principal component 4', 'principal component 5', 'principal component 6',
                          'principal component 7'])
principalDf.head()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7
0,-0.077461,0.015356,0.024770,0.013044,0.025407,-0.004188,-0.003160
1,-0.080020,0.011495,0.015151,0.018436,0.030054,-0.004208,-0.000513
2,-0.081406,0.012380,0.012494,0.014376,0.024197,-0.004589,0.002654
3,-0.083706,0.012758,0.011382,0.015610,0.016446,-0.002929,-0.002039
4,-0.086048,0.007958,0.013954,0.013736,0.018440,-0.004053,-0.001453


In [25]:
#Univariate feature selection
#def univariate_feature_selector(data, k=7):
#    uni_select = SelectKBest(chi2, k)
#    uni_select.fit_transform(data,data.Median_Listing_Price___All_Homes)
#    return data[data.columns[uni_select.get_support(indices=True)]]

#house_uni = univariate_feature_selector(scaled_data, k=7)

#Feature importance based off of gbr
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor

gbrfit = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls').fit(train_x, train_y)
model = SelectFromModel(gbrfit, prefit=True)
#train_x = model.transform(train_x)
print(model.feature_importances_)

NameError: name 'train_x' is not defined

In [65]:
# Split data into test and training data with a test size of 30% (.3)
from sklearn.model_selection import train_test_split # typically done at the start
train_x, test_x, train_y, test_y = train_test_split(
    scaled_data, # features
    outcomes, # outcome
    random_state = 11,
    test_size=0.3 # percentage of data to use as the test set
)

ValueError: Found input variables with inconsistent numbers of samples: [76, 109]

Visualization

In [ ]:
plt.barh(sj_score.model, sj_score.sj_score)
plt.ylabel("Model Names")
plt.xlabel("Negative mean absolute error score")
plt.title("Regressor Models and their error score for San Juan prediction")